# Eval

In [ ]:
#| default_exp eval

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2
# %load_ext rich

In [ ]:
#| export
import os
import json
import yaml
from stringdale.stream_warping import (
    TestCase,
    parse_test_case,
    TraceLog,
    event_stream_warp,
    word_overlap,
    regex,
)

from pathlib import Path
from frozendict import frozendict
from stringdale.core import  checkLogs,await_all
import pytest
import asyncio
from pydantic import BaseModel, ConfigDict

from typing import List, Union
import jsonlines
import logging


In [ ]:
#| export
logger = logging.getLogger(__name__)

## Using podtw

In [ ]:
#| export

def parse_trace_log(trace_path:Union[str,Path]) -> TraceLog:
    """
    Parse a trace file into a list of Trace objects.
    """
    with open(trace_path) as f:
        traces = list(yaml.safe_load_all(f))
        return TraceLog(steps=traces)

In [ ]:
from stringdale.core import get_git_root

In [ ]:
sample_data_dir = get_git_root() / "sample_data" / "eval"

example_case_path = sample_data_dir / "hello_world.yaml"
example_trace_log_path = sample_data_dir / "hello_world.log.yaml"


example_eval_funcs = {
    "word_overlap":word_overlap,
    "regex":regex,
}

In [ ]:
example_test_case = parse_test_case(example_case_path)
example_trace_log  = parse_trace_log(example_trace_log_path)

In [ ]:
best_match,score,debug_info =await event_stream_warp(example_trace_log,example_test_case,eval_funcs=example_eval_funcs,default_func="word_overlap")
best_match

frozendict.frozendict({'3': 2, 'node_a1': 1, 'node_z': 4, '1': 5})

In [ ]:
# debug_info

## Realistic Comparison Functions

In [ ]:
#| export
import numpy as np
import asyncio
from stringdale.db import openai_embed
from stringdale.chat import Chat
from typing import Any

In [ ]:
#| export
async def cosine_dist(out: str, expected: str, model: str = 'text-embedding-3-small') -> float:
    """Compute cosine distance between two strings using OpenAI embeddings.
    
    This function converts two strings into embeddings using OpenAI's embedding model
    and computes the cosine distance between them. The distance is normalized by the norm of the expected embedding.
    
    Args:
        out (str): First string to compare
        expected (str): Second string to compare
        model (str, optional): OpenAI embedding model to use. Defaults to 'text-embedding-3-small'
        
    Returns:
        float: Cosine distance between the strings
        If out is not a string, returns inf
    Raises:
        ValueError: If expected is not a string
        
    Example:
        >>> dist = await cosine_dist("The sky is blue", "The sky is very blue")
        >>> print(dist)  # Returns a small value close to 0
        
    Note:
        The function returns 1 minus the cosine similarity to convert it to a distance
        measure where smaller values indicate more similarity.
    """
    # Get embeddings for both strings
    if not isinstance(out,str):
        return np.inf
    if not isinstance(expected,str):
        raise ValueError(f"cosine_dist: expected is not a string: {expected}")
    out_embedding = await openai_embed(out, model=model)
    expected_embedding = await openai_embed(expected, model=model)
    
    # Compute cosine similarity
    dot_product = np.dot(out_embedding, expected_embedding)
    norm_out = np.linalg.norm(out_embedding)
    norm_expected = np.linalg.norm(expected_embedding)
    
    # Return cosine similarity
    return 1-dot_product / (norm_out * norm_expected)



In [ ]:
basic_dist = await cosine_dist("hello","hello")
basic_dist
assert basic_dist < 0.0001


In [ ]:
await cosine_dist("hello","hello stranger")

np.float64(0.39461446162964386)

In [ ]:
#| export
async def eval_any(out: Any, expected: Any) -> float:
    """Accept any value by always returning 0 distance.
    
    This function is used to verify the existence of a key or value without
    caring about its actual content. It always returns 0, indicating a perfect match
    regardless of the input values.
    
    Args:
        out (Any): First value (ignored)
        expected (Any): Second value (ignored)
        
    Returns:
        float: Always returns 0, indicating a perfect match
    """
    return 0



In [ ]:
#|export
from stringdale.core import jinja_undeclared_vars
from typing import Any

In [ ]:
#| export
class ChatEvalScore(BaseModel):
    score:float


async def chat_eval(out:Any,expected:Any,model:str="gpt-4o-mini",system_prompt:str=None)->float:
    """Evaluate similarity between two values using a language model.
    
    This function uses a language model to compare two values and return a similarity score.
    It can use either a default system prompt or a custom one to guide the evaluation.
    
    Args:
        out (Any): First value to compare
        expected (Any): Second value to compare
        model (str, optional): Language model to use. Defaults to "gpt-4o-mini"
        system_prompt (str, optional): Custom system prompt for evaluation. Must contain
            {{out}} and {{expected}} jinja variables. If None, uses default prompt.
            
    Returns:
        float: Similarity score between 0 and 1, where:
            - 0 indicates lowest similarity
            - 1 indicates highest similarity
            
    Raises:
        ValueError: If system_prompt doesn't contain required jinja variables
        
    Example:
        >>> result = await chat_eval("hello", "world", system_prompt='''
        ...     if one of the strings contains "hello", return 0.5
        ...     string1: {{out}}
        ...     string2: {{expected}}
        ... ''')
        >>> assert result == 0.5  # Returns 0.5 since "hello" is present
    """
    if system_prompt is None:
        system_prompt = """
            You are a helpful assistant that evaluates the similarity of two strings.
            You will be given two strings, and you will need to evaluate the similarity of the two strings.
            You will need to return a score between 0 and 1, where 0 is the lowest similarity and 1 is the highest similarity.

            string1: {{out}}
            string2: {{expected}}

            return a score between 0 and 1, where 0 is the lowest similarity and 1 is the highest similarity.
            """

    if not jinja_undeclared_vars(system_prompt) == {'out','expected'}:
        raise ValueError("System prompt must contain {{out}} and {{expected}} jinja variables")

    chat = Chat(model=model,messages=
        [{"role":"system","content":system_prompt}],
        output_schema=ChatEvalScore,
        out = out,
        expected = expected,
        )
    response = await chat()
    return response['content'].score

In [ ]:
custom_prompt = """
if one of the strings contains "hello", return 0.5

string1: {{out}}
string2: {{expected}}
"""


result = await chat_eval("hello","world",system_prompt=custom_prompt)
assert result == 0.5

In [ ]:
custom_prompt = """
if one of the strings contains "hello", return 0.5

string1: {{outs}}
string2: {{expected}}
"""

with pytest.raises(ValueError,match="System prompt must contain {{out}} and {{expected}} jinja variables"):
    result = await chat_eval("hello","world",system_prompt=custom_prompt)


In [ ]:
#| export
def eq(a,b):
    """Compare two values for exact equality.
    
    Args:
        a: First value to compare
        b: Second value to compare
        
    Returns:
        float: Returns 0 if values are equal, infinity if they differ
        
    Example:
        >>> eq(5, 5)  # Returns 0
        >>> eq("hello", "world")  # Returns inf
    """
    if a == b:
        return 0
    else:
        return np.inf


In [ ]:
out = 3500
expr = "({0} < 4000) & ({0} > 3000)"

In [ ]:
f_expr = expr.format(out)
f_expr

'(3500 < 4000) & (3500 > 3000)'

In [ ]:
#| export
from stringdale.tools import run_python_code


In [ ]:
#| export
def safe_eval(out,expression):
    """Safely evaluate a Python expression with a provided value.
    
    This function takes a value and a Python expression template, formats the expression
    with the value, and evaluates it safely. It handles various error cases and type
    conversions.
    
    Args:
        out: Value to insert into the expression template
        expression (str): Python expression template with {} placeholder for the value
        
    Returns:
        float: 
            - For boolean results: 0 if True, infinity if False
            - For float results: the float value directly
            - For errors or invalid types: infinity
            
    Example:
        >>> safe_eval(5, "{0} < 10")  # Returns 0 (True)
        >>> safe_eval(15, "{0} < 10")  # Returns inf (False)
        >>> safe_eval(3.5, "2 * {0}")  # Returns 7.0
    
    Note:
        - Uses run_python_code for safe evaluation
        - Logs warnings for formatting and evaluation errors
        - Logs debug info for successful evaluations
    """
    try:
        formatted_expressions = expression.format(out)
    except Exception as e:
        logger.warning(f"Error formatting expression: {expression} with value {out}, error: {e}")
        return np.inf
    value = run_python_code(formatted_expressions)
    if isinstance(value,str) and value.startswith("Error"):
        logger.warning(
            f"Error evaluating expression: {formatted_expressions} = {value}\n"
            f"out: {out}\n"
            f"expression: {expression}\n"
            f"error: {e}"
        )
        return np.inf
    logger.debug(f"safe_eval: {formatted_expressions} = {value}")
    if isinstance(value,bool):
        return 0 if value else np.inf
    elif isinstance(value,float):
        return value
    else:
        logger.debug(
            f"When evaluating {expression} with value {out}\n"
            f"Expected float or bool, got {type(value)} with value {repr(value)}"
            )
        return np.inf

In [ ]:
eval_string ="""
x=4000
({0} < x) & ({0} > 3000)
"""

with checkLogs():
    y =safe_eval(3500,eval_string)
y

__main__ - DEBUG - safe_eval: 
x=4000
(3500 < x) & (3500 > 3000)
 = True


0

In [ ]:
safe_eval(3500,"""
x=4000
({0} < x) & ({0} > 3000)
""")

0

## Running and evaluating a single data point

In [ ]:
#| export
from typing import List,Dict,Callable

In [ ]:
#| export
class DataPoint(BaseModel):
    traces:TraceLog
    expected:TestCase
    

In [ ]:
#| export

async def _run_agent(Agent,test_case:TestCase,trace_log_path:Path):
    d=Agent()
    if trace_log_path.exists():
        os.unlink(trace_log_path)
    for input in test_case.inputs:
        async for trace in d.arun(input):
            trace.pprint(skip_passthrough=True,add_keys=['duration'],drop_keys=['input'],file=trace_log_path)
        if d.finished:
            break

async def evaluate_datapoint(Agent,eval_funcs,default_func,test_case_path,trace_log_path=None,force_run=False):
    if trace_log_path is None:
        trace_log_path = test_case_path.parent/test_case_path.name.replace(".yaml", ".log.yaml")

    if not trace_log_path.parent.exists():
        os.makedirs(trace_log_path.parent,exist_ok=True)
    try:
        test_case = parse_test_case(test_case_path)
    except Exception as e:
        raise ValueError(f"Error parsing test case {test_case_path}: {e}") from e
        

    if force_run or not trace_log_path.exists():
        if not trace_log_path.exists():
            logger.info(f"Trace file {trace_log_path.name} does not exist, running agent")
        else:
            logger.info(f"Force running {trace_log_path.name}")
        await _run_agent(Agent,test_case,trace_log_path)
    else:
        logger.info(f"Trace file {trace_log_path.name} already exists, skipping agent run")

    parsed_trace = parse_trace_log(trace_log_path)
    aligned_trace,score,debug_info = await event_stream_warp(parsed_trace,test_case,eval_funcs,default_func)
    
    return aligned_trace,score,debug_info,trace_log_path


In [ ]:
from stringdale.examples.react import ReactAgent

In [ ]:
agent = ReactAgent
expected_yaml = sample_data_dir/"obama.yaml"
bad_expected_yaml = sample_data_dir/"react_bad_expected.yaml"
eval_funcs = {
    "eq":eq,
    "eval":safe_eval,
    "cosine_dist":cosine_dist,
}
default_func = 'cosine_dist'


In [ ]:

with checkLogs():
    alignment,score,debug_info,trace_out = await evaluate_datapoint(agent,eval_funcs,default_func,bad_expected_yaml)

assert alignment is None
alignment,score,trace_out


__main__ - INFO - Trace file react_bad_expected.log.yaml already exists, skipping agent run
No viable trace row nums for expected trace 1
No possible mappings found


(None,
 inf,
 PosixPath('/Users/dean/dl/stringdale/sample_data/eval/react_bad_expected.log.yaml'))

In [ ]:
with checkLogs(level='INFO'):
    alignment,score,debug_info,trace_out = await evaluate_datapoint(agent,eval_funcs,default_func,expected_yaml)

assert dict(alignment) == {'0': 1, '1': 7}, dict(alignment)
alignment,score,trace_out

__main__ - INFO - Trace file obama.log.yaml does not exist, running agent


(frozendict.frozendict({'0': 1, '1': 7}),
 np.float64(0.3237201985261057),
 PosixPath('/Users/dean/dl/stringdale/sample_data/eval/obama.log.yaml'))

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export

def _pd_order_columns_first(df:pd.DataFrame,first_columns:list[str]):
    """
    Reorder the columns of a pandas dataframe to put the first_columns first.
    """
    return df[first_columns + [c for c in df.columns if c not in first_columns]]



In [ ]:
x = pd.DataFrame([
    {'distance':1,'comparison':'eq','actual':1,'expected':1},
    {'distance':2,'comparison':'eq','actual':2,'expected':2},
    {'distance':3,'comparison':'eq','actual':3,'expected':3},
])

_pd_order_columns_first(x,['actual','expected'])

,actual,expected,distance,comparison
0,1,1,1,eq
1,2,2,2,eq
2,3,3,3,eq


In [ ]:
# debug_info

In [ ]:
#| export
from copy import deepcopy
from itertools import count


In [ ]:
#| export

def summarize_datapoint(name,alignment,debug_info):
    """
    Summarize the datapoint by getting the distance per step and total metrics such as sum of distances and coverage
    by using the alignment and the debug info
    """
    deep_dive_fit = []

    comp_counter = count()
    for expected_node_id,trace_idx in alignment.items():
        match_data = debug_info[expected_node_id][trace_idx]
        for comp in match_data['comparisons']:
            summary = deepcopy(match_data) | deepcopy(comp) 
            summary['comp_id'] = next(comp_counter)
            summary.pop('comparisons')
            summary['aggregation'] = comp['aggregation']
            deep_dive_fit.append(summary)


    df = pd.DataFrame(deep_dive_fit)
    df['datapoint'] = str(name)
    df = _pd_order_columns_first(df,['datapoint','node_label','trace_idx','func','key','actual','expected','distance'])
    return df

In [ ]:
df = summarize_datapoint('react',alignment,debug_info)
df

,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,react,0,1,eq,content.name,wikipedia_search,wikipedia_search,0.00000,0,use_tool,use_tool,{},None,0
1,react,0,1,None,content.input.q,Barack Obama,Obama,0.32372,0,use_tool,use_tool,{},None,1
2,react,1,7,eq,content.name,run_python_code,run_python_code,0.00000,1,use_tool,use_tool,{},None,2
3,react,1,7,eval,content.output,3844,({0} < 4000) & ({0} > 3000),0.00000,1,use_tool,use_tool,{},None,3


In [ ]:
assert df['node_label'].to_list() == ['0','0','1','1']
assert df['key'].to_list() == ['content.name','content.input.q','content.name','content.output']

In [ ]:
#| export
from stringdale.stream_warping import parse_test_case

## DataSets

In [ ]:
#| export
def filter_and_concat(df1: pd.DataFrame, df2: pd.DataFrame, keys: list) -> pd.DataFrame:
    """
    Filter df1 by removing rows with matching key values in df2, then concatenate with df2.
    
    Args:
        df1 (pd.DataFrame): First DataFrame to filter
        df2 (pd.DataFrame): Second DataFrame to concatenate
        keys (list): List of column names to use as keys for matching
        
    Returns:
        pd.DataFrame: Concatenated DataFrame with filtered df1 and df2
    """
    if df1.empty:
        return df2
    if df2.empty:
        return df1
    # Create tuples of key values for comparison
    mask = df1[keys].apply(tuple, axis=1).isin(df2[keys].apply(tuple, axis=1))
    
    # Filter df1 to keep only rows that don't exist in df2 (using inverse mask)
    df1_filtered = df1[~mask]
    
    # Concatenate the filtered df1 with df2
    result = pd.concat([df1_filtered, df2], ignore_index=True)
    
    return result

In [ ]:
# Example DataFrames
df1 = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': ['a', 'b', 'c', 'd'],
    'C': [10, 20, 30, 40]
})

df2 = pd.DataFrame({
    'A': [3, 4],
    'B': ['c', 'd'],
    'C': [50, 60]
})

# List of keys to match on
keys = ['A', 'B']

# Apply the function
result = filter_and_concat(df1, df2, keys)

assert result.equals(pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': ['a', 'b', 'c', 'd'],
    'C': [10, 20, 50, 60]
}))
print(result)

   A  B   C
0  1  a  10
1  2  b  20
2  3  c  50
3  4  d  60


In [ ]:
#| export
from stringdale import DiagramSchema
from pprint import pprint, pformat
from fastcore.basics import patch
from typing import Optional
from pydantic import BaseModel, ConfigDict, PrivateAttr
import pandas as pd
import json


In [ ]:
#| export

# Define a JSON parser function that handles potential errors
def parse_json(data):
    try:
        # Handle cases where JSON might use single quotes instead of double quotes
        if isinstance(data, str):
            data = data.replace("'", '"')
        return json.loads(data)
    except (json.JSONDecodeError, TypeError):
        return None

In [ ]:
#| export
class TestSetRun(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    # Private attributes
    _summary_dict: dict = PrivateAttr(default_factory=dict)
    _details_dict: dict = PrivateAttr(default_factory=dict)
    
    # Regular fields
    test_dir: Path
    dir: Path
    summary: pd.DataFrame
    details: pd.DataFrame
    debug: dict

    def find_cases(self):
        yaml_paths =  list(self.test_dir.glob("**/*.yaml"))
        return [str(p.relative_to(self.test_dir).with_suffix("")) for p in yaml_paths]

    def trace_log_path(self,datapoint:str):
        return self.dir/'logs'/f'{datapoint}.log.yaml'

    def trace_log_len(self,datapoint:str):
        log_path = self.trace_log_path(datapoint)
        return len(log_path.read_text().splitlines())

    def testcase_path(self,datapoint:str):
        return self.test_dir/f'{datapoint}.yaml'

    def serialize_test_case(self,datapoint:str):
        yml_version = yaml.safe_load(self.testcase_path(datapoint).read_text())
        json_version = json.dumps(yml_version,indent=2)
        return json_version

    def datapoint_len(self,datapoint:str):
        datapoint_yaml = (self.test_dir/datapoint).with_suffix(".yaml")
        return trace_log_len(self.dir/datapoint_yaml)
        
    def __repr__(self):     
        return (
            f"TestSetRun(\n"
            f"  test_dir={self.test_dir}, \n"
            f"  dir={self.dir}, \n"
            f"  summary=Dataframe({self.summary.shape}), \n"
            f"  details=Dataframe({self.details.shape}), \n"
            f"  debug=dict)")

    def __str__(self):
        return self.__repr__()

    def save(self,dir:Path):
        self.summary.to_csv(dir/"summary.csv",index=False)
        self.details.to_csv(dir/"details.csv",index=False)
        test_dir_rel = Path(os.path.relpath(self.test_dir,self.dir))
        (dir/'test_cases_loc.txt').write_text(str(test_dir_rel))
        with open(dir/"debug.json","w") as f:
            json.dump(self.debug,f)

    @classmethod
    def load(cls, dir: Path,test_dir:Optional[Path]=None):
        # Initialize empty DataFrames and dict for missing files
        summary = pd.DataFrame()
        details = pd.DataFrame()
        debug = {}
        if test_dir is None:
            test_dir = dir
        
        # Try to load files if they exist
        try:
            if (dir/"summary.csv").exists():
                summary = pd.read_csv(dir/"summary.csv",index_col=False)
            if (dir/"details.csv").exists():
                details = pd.read_csv(dir/"details.csv",index_col=False,converters={'kwargs':parse_json})
            if (dir/"debug.json").exists():
                with open(dir/"debug.json") as f:
                    debug = json.load(f)
            if (dir/'test_cases_loc.txt').exists():
                test_cases_loc = (dir/'test_cases_loc.txt').read_text().strip()
                test_cases_loc = dir/test_cases_loc
        except Exception as e:
            # Log the error but continue with empty/default values
            print(f"Warning: Error loading some files: {str(e)}")
        
        return cls(
            test_dir=test_dir,
            dir=dir,
            summary=summary,
            details=details,
            debug=debug
        )

    def is_datapoint_stale(self,datapoint_path):
        if self.summary.empty:
            return True
        datapoints  = self.summary['datapoint'].unique().tolist()
        # if the datapoint is not in the summary, it is stale
        if datapoint_path not in datapoints:
            return True
        
        summarized_test_case = self.summary.loc[self.summary['datapoint'] == datapoint_path]['serialized_test_case'].iloc[0]
        current_test_case = self.serialize_test_case(datapoint_path)
        return summarized_test_case != current_test_case
        


In [ ]:
eval_ds_dir = get_git_root() / "sample_data" / "eval_datasets"
test_dir = eval_ds_dir / "test_cases"
out_dir = get_git_root() / "logs" / "testset_out"


In [ ]:
run = TestSetRun.load(out_dir,test_dir=test_dir)
cases = run.find_cases()
assert cases == ['pikachus','goldens'],cases

In [ ]:
run.trace_log_path('pikachus').relative_to(run.dir)

PosixPath('logs/pikachus.log.yaml')

In [ ]:
run.summary

""


In [ ]:
run.details

""


In [ ]:
#| export

async def eval_dataset(Agent:DiagramSchema,test_dir,out_dir,eval_funcs,default_func,force_run=False):

    run = TestSetRun.load(out_dir)
    run.test_dir = test_dir    
    datapoints = run.find_cases()

    if not force_run:
        stale_datapoints = [p for p in datapoints if run.is_datapoint_stale(p)]
    else:
        stale_datapoints = datapoints

    if len(stale_datapoints) > 0:
        logger.info(f"{run.dir.name}: Evaluating {len(stale_datapoints)}/{len(datapoints)} datapoints")
    else:
        logger.info(f"{run.dir.name}: No stale datapoints, skipping evaluation")
        return run
    
    datapoint_results = await await_all(
        [
            evaluate_datapoint(
                Agent=Agent,
                # TODO make sure we use the eval func syntax throughout the whole module
                eval_funcs=eval_funcs,
                default_func=default_func,
                test_case_path=run.testcase_path(datapoint),
                trace_log_path=run.trace_log_path(datapoint),
                force_run=True, # since we computed which datapoints to run, we can force run them
            ) for datapoint in stale_datapoints
        ],
        error_prefix=[
            f"When evaluating datapoint {datapoint}"
            for datapoint in stale_datapoints
        ]
    )
    
    summary_data = list()
    deep_dives = list()
    debug_infos = dict()

    for (alignment,score,debug_info,trace_out),datapoint in zip(datapoint_results,stale_datapoints):
        debug_infos[datapoint] = debug_info
        deep_dive = summarize_datapoint(datapoint,alignment,debug_info)
        deep_dive['datapoint'] = datapoint
        deep_dives.append(deep_dive)
        summary_data.append({
            'datapoint':str(datapoint),
            'distance':score,
            'avg_distance':deep_dive.distance.mean(),
            'coverage':len(alignment) / run.trace_log_len(datapoint),
            'alignment':alignment,
            'serialized_test_case':run.serialize_test_case(datapoint),
            })

    
    new_summary = pd.DataFrame.from_records(summary_data).reset_index(drop=True)
    run.summary = filter_and_concat(run.summary,new_summary,['datapoint'])

    details_data = pd.concat(deep_dives,ignore_index=True)
    run.details = filter_and_concat(run.details,details_data,['datapoint'])
    run.debug = {**run.debug,**debug_infos}
    run.save(out_dir)
    
    return run

In [ ]:
from stringdale.examples.rag import Rag
from stringdale.db import ChromaClient
import tempfile

In [ ]:

def load_agent(conf_dir: Path):
    agent_yaml_path = conf_dir / 'agent.yml'
    vec_db_yaml_path = conf_dir / 'vec_db.yml'

    agent_conf = yaml.safe_load(agent_yaml_path.read_text())
    vec_db_conf = yaml.safe_load(vec_db_yaml_path.read_text())

    db = ChromaClient(persist_path=tempfile.mkdtemp())
    for collection_name, docs in vec_db_conf.items():
        db.add_collection(collection_name, exists_ok=True)
        db.upsert(collection_name, docs)

    agent_conf['db'] = db
    
    Agent = Rag(**agent_conf)

    return Agent

In [ ]:
conf_dir = eval_ds_dir / "agent_configs"

eval_funcs = {
    'eq':eq,
    'eval':safe_eval,
    'chat_eval':chat_eval,
    'cosine_dist':cosine_dist,
}

default_func = 'cosine_dist'

In [ ]:
with checkLogs(level='INFO'):
    run1 = await eval_dataset(
        Agent=load_agent(conf_dir/'v001'),
        test_dir=test_dir,
        out_dir=out_dir/'v001',
        eval_funcs=eval_funcs,
        default_func=default_func,
        )

    run2 = await eval_dataset(
        Agent=load_agent(conf_dir/'v002'),
        test_dir=test_dir,
        out_dir=out_dir/'v002',
        eval_funcs=eval_funcs,
        default_func=default_func)

    run3 = await eval_dataset(
        Agent=load_agent(conf_dir/'v003'),
        test_dir=test_dir,
        out_dir=out_dir/'v003',
        eval_funcs=eval_funcs,
        default_func=default_func)



__main__ - INFO - v001: No stale datapoints, skipping evaluation
__main__ - INFO - v002: No stale datapoints, skipping evaluation
__main__ - INFO - v003: No stale datapoints, skipping evaluation


In [ ]:
run1.summary

,datapoint,distance,avg_distance,coverage,alignment,serialized_test_case
0,pikachus,0.806506,0.806506,0.022727,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ..."
1,goldens,0.766294,0.383147,0.044444,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ..."


In [ ]:
run1.details

,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,pikachus,0,1,NaN,content,Pikachus are not dogs; they are fictional crea...,They are:\n * dangerous\n * smoke cigarettes,0.806506,0,chat,chat,{},NaN,0
1,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.666294,1,chat,chat,{},NaN,1


In [ ]:
run2.details

,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,pikachus,0,1,NaN,content,Pikachu is a fictional character from the Poké...,They are:\n * dangerous\n * smoke cigarettes,0.877153,0,chat,chat,{},NaN,0
1,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,goldens,1,1,NaN,content,"Golden Retrievers are a friendly, intelligent ...",They are:\n * friendly\n * intelligent\n,0.628687,1,chat,chat,{},NaN,1


In [ ]:
run3.details

,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,pikachus,0,1,NaN,content,Pikachus are dangerous creatures that smoke to...,They are:\n * dangerous\n * smoke cigarettes,0.443827,0,chat,chat,{},NaN,0
1,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.647421,1,chat,chat,{},NaN,1


## Comparing Runs

In [ ]:
#| export
import math
from typing import Optional
import textwrap

In [ ]:
#| export
class Comparison(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    from_run: TestSetRun
    to_run: TestSetRun
    dir: Optional[Path]
    summary: pd.DataFrame
    details: pd.DataFrame

    def __repr__(self):     
        return (
            f"Comparison(\n"
            f"  from_run={textwrap.indent(self.from_run.__repr__(), '  ').strip()}, \n"
            f"  to_run={textwrap.indent(self.to_run.__repr__(), '  ').strip()}, \n"
            f"  summary=Dataframe({self.summary.shape}), \n"
            f"  details=Dataframe({self.details.shape}), \n"
            f")")

    def __str__(self):
        return self.__repr__()

    def save(self):
        if self.dir is None:
            return
        self.dir.mkdir(parents=True,exist_ok=True)
        self.summary.to_csv(self.dir/"summary.csv",index=False)
        self.details.to_csv(self.dir/"details.csv",index=False)
    


In [ ]:
#| export
def sort_conditions(df):
    return df.sort_values(by=['comp_id'])

def limit_to_datapoint(df,datapoint):
    return df.loc[df['datapoint'] == datapoint]

def get_datapoint(ds,datapoint):
    return sort_conditions(limit_to_datapoint(ds.details,datapoint))


def describe_changes(ds1,ds2,datapoint,epsilon=1e-3):
    """
    Describe the changes between two datapoints
    """
    
    # get the detailed version of the datasets and limit to only rows of the given datapoint
    datapoint_df1 = get_datapoint(ds1,datapoint)
    datapoint_df2 = get_datapoint(ds2,datapoint)

    # since these datapoints or not extended or reduced, we expect the same set of expected nodes and the same set of tuples of the type (content,comparison)
    # lets assert this in the code
    assert datapoint_df1.shape == datapoint_df2.shape, f"Datapoint {datapoint} has different number of rows in the two datasets {ds1} and {ds2}"

    changes = []

    # get the first comparison whose node aligned to a different trace
    for row1,row2 in zip(datapoint_df1.itertuples(),datapoint_df2.itertuples()):
        if row1.trace_idx != row2.trace_idx:
            changes.append({
                'datapoint':datapoint,
                'change_type':'alignment_change',
                'before':row1.trace_idx,
                'after':row2.trace_idx,
                'comparison_id':row1.comp_id,
            })
            break
            

    for row1,row2 in zip(datapoint_df1.itertuples(),datapoint_df2.itertuples()):
        if math.isclose(row1.distance,row2.distance,abs_tol=epsilon):
            continue
        if row2.distance + epsilon > row1.distance:
            change_types = 'regressed'
        elif row2.distance - epsilon < row1.distance:
            change_types = 'improved'
        else:
            continue
        
        changes.append({
            'datapoint':datapoint,
            'change_type':change_types,
            'value':row1.distance - row2.distance,
            'comp_id':row1.comp_id,
            'node_label':row1.node_label,
            'expected':row1.expected,
            'before':row1.actual,
            'after':row2.actual,
        })
    
    return pd.DataFrame(changes)


            

In [ ]:
describe_changes(run1,run2,'goldens')

,datapoint,change_type,value,comp_id,node_label,expected,before,after
0,goldens,improved,0.037607,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are a friendly, intelligent ..."


In [ ]:
#| export
def compare_datasets(ds1,ds2,epsilon=1e-3,out_dir=None):
    """
    Compare two datasets
    """
    
    summary_1 = ds1.summary.sort_values(by='datapoint')
    summary_2 = ds2.summary.sort_values(by='datapoint')

    changed_datapoints = []
    change_summary = []
    detailed_changes = []

    for row1,row2 in zip(summary_1.itertuples(),summary_2.itertuples()):
        datapoint = row1.datapoint
        distance_change = not math.isclose(row1.distance,row2.distance,abs_tol=epsilon)
        coverage_change = row1.coverage != row2.coverage

        if distance_change or coverage_change:
            changed_datapoints.append(datapoint)

            detailed_change = describe_changes(ds1,ds2,datapoint,epsilon)
            detailed_changes.append(detailed_change)

            change_types = set(detailed_change['change_type'])
            if 'alignment_change' in change_types:
                alignment_change = True
            else:
                alignment_change = False
            
            if 'improved' in change_types and not 'regressed' in change_types:
                score_change = 'improved'
            elif 'regressed' in change_types and not 'improved' in change_types:
                score_change = 'regressed'
            else:
                score_change = 'changed'
            
            total_score_change = row1.distance-row2.distance

            change_summary.append({
                'datapoint':datapoint,
                'alignment_change':alignment_change,
                'score_change_type':score_change,
                'total_score_change':total_score_change,
            })

    changes_summary = pd.DataFrame(change_summary)
    if len(detailed_changes) > 0:
        detailed_changes = pd.concat(detailed_changes)
    else:
        detailed_changes = pd.DataFrame()

    comp =  Comparison(
        from_run=ds1,
        to_run=ds2,
        summary=changes_summary,
        details=detailed_changes,
        dir=out_dir,
    )
    comp.save()
    return comp

In [ ]:
comparison_2 = compare_datasets(run1,run2)
comparison_2

Comparison(
  from_run=TestSetRun(
    test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
    dir=/Users/dean/dl/stringdale/logs/testset_out/v001, 
    summary=Dataframe((2, 6)), 
    details=Dataframe((3, 14)), 
    debug=dict), 
  to_run=TestSetRun(
    test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
    dir=/Users/dean/dl/stringdale/logs/testset_out/v002, 
    summary=Dataframe((2, 6)), 
    details=Dataframe((3, 14)), 
    debug=dict), 
  summary=Dataframe((2, 4)), 
  details=Dataframe((2, 8)), 
)

In [ ]:
comparison_2.summary

,datapoint,alignment_change,score_change_type,total_score_change
0,goldens,False,improved,0.037607
1,pikachus,False,regressed,-0.070646


In [ ]:
comparison_2.details

,datapoint,change_type,value,comp_id,node_label,expected,before,after
0,goldens,improved,0.037607,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are a friendly, intelligent ..."
0,pikachus,regressed,-0.070646,0,0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachu is a fictional character from the Poké...


In [ ]:
comparison_3 = compare_datasets(run1,run3)
comparison_3.summary


,datapoint,alignment_change,score_change_type,total_score_change
0,goldens,False,improved,0.018873
1,pikachus,False,improved,0.362679


In [ ]:
comparison_3.details

,datapoint,change_type,value,comp_id,node_label,expected,before,after
0,goldens,improved,0.018873,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","The Golden Retriever is a friendly, intelligen..."
0,pikachus,improved,0.362679,0,0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachus are dangerous creatures that smoke to...


In [ ]:
run1

TestSetRun(
  test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
  dir=/Users/dean/dl/stringdale/logs/testset_out/v001, 
  summary=Dataframe((2, 6)), 
  details=Dataframe((3, 14)), 
  debug=dict)

In [ ]:
# then continue to make the pprint of a change class with topk changes of each type

## EvalResult

In [ ]:
#| export
from typing import Callable,Dict,List,Optional,Tuple

In [ ]:
#| export
class EvalResult():
    """
    A class to track evaluation results, including individual runs and comparisons between runs.
    
    Attributes:
        runs (Dict[str, TestSetRun]): Dictionary mapping agent names to their test run results
        comparisons (Dict[Tuple[str, str], Comparison]): Dictionary mapping pairs of agent names 
            (base_run, other_run) to their comparison results
        eval_funcs (Dict[str, Callable]): Dictionary mapping eval function names to their implementations
        default_func (Callable): The default eval function to use if no eval function is specified
        run_summaries (pd.DataFrame): A DataFrame containing summary information for each run
        run_details (pd.DataFrame): A DataFrame containing detailed information for each run
        comp_summaries (pd.DataFrame): A DataFrame containing summary information for each comparison
        comp_details (pd.DataFrame): A DataFrame containing detailed information for each comparison
    """

    def __init__(self,runs:Dict[str,TestSetRun],comparisons:Dict[Tuple[str,str],Comparison],eval_funcs:Dict[str,Callable],default_func:Callable):
        self.runs = runs
        self.comparisons = comparisons
        self.eval_funcs = eval_funcs
        self.default_func = default_func

        run_summaries = []
        run_details = []
        comp_summaries = []
        comp_details = []
        for run_name,run in runs.items():
            run_s = run.summary.copy()
            run_s['agent'] = run_name
            run_s['test_case_path'] = run_s.apply(
                lambda row: os.path.relpath(run.testcase_path(row['datapoint']), os.getcwd()), 
                axis=1
            )
            run_s['trace_log_path'] = run_s.apply(
                lambda row: os.path.relpath(run.trace_log_path(row['datapoint']), os.getcwd()), 
                axis=1
            )
            run_summaries.append(run_s)
            run_d = run.details.copy()
            run_d['agent'] = run_name
            run_details.append(run_d)
        for (base_run,other_run),comparison in comparisons.items():
            comp_s = comparison.summary.copy()
            comp_s['from_agent'] = base_run
            comp_s['to_agent'] = other_run
            comp_summaries.append(comp_s)
            comp_d = comparison.details.copy()
            comp_d['from_agent'] = base_run
            comp_d['to_agent'] = other_run
            comp_details.append(comp_d)

        self.run_summaries = _pd_order_columns_first( pd.concat(run_summaries),['agent'])
        self.run_details = _pd_order_columns_first( pd.concat(run_details),['agent'])
        self.comp_summaries = _pd_order_columns_first( pd.concat(comp_summaries),['from_agent','to_agent'])
        self.comp_details = _pd_order_columns_first( pd.concat(comp_details),['from_agent','to_agent'])



    
    def save(self,out_dir:Path):
        self.run_summaries.to_csv(out_dir/'run_summaries.csv',index=False)
        self.run_details.to_csv(out_dir/'run_details.csv',index=False)
        self.comp_summaries.to_csv(out_dir/'comp_summaries.csv',index=False)
        self.comp_details.to_csv(out_dir/'comp_details.csv',index=False)

    def __repr__(self) -> str:
        runs_str = f"runs: {list(self.runs.keys())}"
        comparisons_str = f"comparisons: {list(self.comparisons.keys())}"
        return f"EvalResult(\n  {runs_str},\n  {comparisons_str}\n)"
    
    def __str__(self) -> str:
        return self.__repr__()

In [ ]:
res = EvalResult(
    runs={
        'v001':run1,
        'v002':run2,
        'v003':run3,
    },
    comparisons={
        ('v001','v002'):comparison_2,
        ('v001','v003'):comparison_3,
    },
    eval_funcs=eval_funcs,
    default_func='cosine_dist',
    )

In [ ]:
res.run_summaries

,agent,datapoint,distance,avg_distance,coverage,alignment,serialized_test_case,test_case_path,trace_log_path
0,v001,pikachus,0.806506,0.806506,0.022727,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v001/logs/pikachus.log.yaml
1,v001,goldens,0.766294,0.383147,0.044444,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v001/logs/goldens.log.yaml
0,v002,pikachus,0.877153,0.877153,0.022222,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v002/logs/pikachus.log.yaml
1,v002,goldens,0.728687,0.364343,0.046512,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v002/logs/goldens.log.yaml
0,v003,pikachus,0.443827,0.443827,0.020833,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v003/logs/pikachus.log.yaml
1,v003,goldens,0.747421,0.373710,0.041667,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v003/logs/goldens.log.yaml


In [ ]:
res.run_details

,agent,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,v001,pikachus,0,1,NaN,content,Pikachus are not dogs; they are fictional crea...,They are:\n * dangerous\n * smoke cigarettes,0.806506,0,chat,chat,{},NaN,0
1,v001,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,v001,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.666294,1,chat,chat,{},NaN,1
0,v002,pikachus,0,1,NaN,content,Pikachu is a fictional character from the Poké...,They are:\n * dangerous\n * smoke cigarettes,0.877153,0,chat,chat,{},NaN,0
1,v002,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,v002,goldens,1,1,NaN,content,"Golden Retrievers are a friendly, intelligent ...",They are:\n * friendly\n * intelligent\n,0.628687,1,chat,chat,{},NaN,1
0,v003,pikachus,0,1,NaN,content,Pikachus are dangerous creatures that smoke to...,They are:\n * dangerous\n * smoke cigarettes,0.443827,0,chat,chat,{},NaN,0
1,v003,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
2,v003,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.647421,1,chat,chat,{},NaN,1


In [ ]:
res.comp_summaries

,from_agent,to_agent,datapoint,alignment_change,score_change_type,total_score_change
0,v001,v002,goldens,False,improved,0.037607
1,v001,v002,pikachus,False,regressed,-0.070646
0,v001,v003,goldens,False,improved,0.018873
1,v001,v003,pikachus,False,improved,0.362679


In [ ]:
res.comp_details

,from_agent,to_agent,datapoint,change_type,value,comp_id,node_label,expected,before,after
0,v001,v002,goldens,improved,0.037607,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are a friendly, intelligent ..."
0,v001,v002,pikachus,regressed,-0.070646,0,0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachu is a fictional character from the Poké...
0,v001,v003,goldens,improved,0.018873,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","The Golden Retriever is a friendly, intelligen..."
0,v001,v003,pikachus,improved,0.362679,0,0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachus are dangerous creatures that smoke to...


In [ ]:
res.run_details.merge(res.comp_details,left_on=['agent','datapoint','comp_id'],right_on=['to_agent','datapoint','comp_id'],how='left')

,agent,datapoint,node_label_x,trace_idx,func,key,actual,expected_x,distance,node_idx,...,aggregation,comp_id,from_agent,to_agent,change_type,value,node_label_y,expected_y,before,after
0,v001,pikachus,0,1,NaN,content,Pikachus are not dogs; they are fictional crea...,They are:\n * dangerous\n * smoke cigarettes,0.806506,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,v001,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,...,min,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,v001,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.666294,1,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,v002,pikachus,0,1,NaN,content,Pikachu is a fictional character from the Poké...,They are:\n * dangerous\n * smoke cigarettes,0.877153,0,...,NaN,0,v001,v002,regressed,-0.070646,0.0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachu is a fictional character from the Poké...
4,v002,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,...,min,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,v002,goldens,1,1,NaN,content,"Golden Retrievers are a friendly, intelligent ...",They are:\n * friendly\n * intelligent\n,0.628687,1,...,NaN,1,v001,v002,improved,0.037607,1.0,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are a friendly, intelligent ..."
6,v003,pikachus,0,1,NaN,content,Pikachus are dangerous creatures that smoke to...,They are:\n * dangerous\n * smoke cigarettes,0.443827,0,...,NaN,0,v001,v003,improved,0.362679,0.0,They are:\n * dangerous\n * smoke cigarettes,Pikachus are not dogs; they are fictional crea...,Pikachus are dangerous creatures that smoke to...
7,v003,goldens,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,...,min,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,v003,goldens,1,1,NaN,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.647421,1,...,NaN,1,v001,v003,improved,0.018873,1.0,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","The Golden Retriever is a friendly, intelligen..."


In [ ]:
#| export
import rich
from rich.padding import Padding
from stringdale.core import jinja_render

In [ ]:
#| export
def rprint(obj,indent:int=0,sep_by:int=2):
    rich.print(Padding(obj,pad=(0,0,0,indent*sep_by)))

In [ ]:
#| export
def _pprint_datapoint_data_prep(res:EvalResult,datapoint:str,default_comparison="cosine_dist"):
    
    base_name = list(res.comparisons.keys())[0][0]
    test_case_loc = res.runs[base_name].testcase_path(datapoint)
    test_case_loc = os.path.relpath(test_case_loc,os.getcwd())

    run_sum = res.run_summaries[res.run_summaries['datapoint'] == datapoint]
    comp_sum = res.comp_summaries[res.comp_summaries['datapoint'] == datapoint]
    run_det = res.run_details[res.run_details['datapoint'] == datapoint]
    comp_det = res.comp_details[res.comp_details['datapoint'] == datapoint]

    run_det_with_comp = run_det.merge(
        comp_det[['to_agent','datapoint','comp_id','change_type','value']],
        left_on=['agent','datapoint','comp_id'],
        right_on=['to_agent','datapoint','comp_id'],
        how='left').fillna(value='None')

    per_comp = {
        int(comp_id):
        (
        run_det[(run_det['agent'] == base_name) & (run_det['comp_id'] == comp_id)].iloc[0],
        run_det_with_comp[run_det_with_comp['comp_id'] == comp_id]
        )
        for comp_id in comp_det['comp_id'].unique()
    }

    jinja_params = {
        'base_name':base_name,
        'test_case_loc':test_case_loc,
        'datapoint':datapoint,
        'run_sum':run_sum,
        'comp_sum':comp_sum,
        'per_comp':per_comp,

        'version_style':'purple',
        'param_style':'cyan bold',
        'comp_config_style':'green bold',
        'output_style':'#CE9178',
    }
    return jinja_params
    

In [ ]:
params = _pprint_datapoint_data_prep(res,'pikachus')
params['per_comp'][0][1]

,agent,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id,to_agent,change_type,value
0,v001,pikachus,0,1,None,content,Pikachus are not dogs; they are fictional crea...,They are:\n * dangerous\n * smoke cigarettes,0.806506,0,chat,chat,{},None,0,None,None,None
1,v002,pikachus,0,1,None,content,Pikachu is a fictional character from the Poké...,They are:\n * dangerous\n * smoke cigarettes,0.877153,0,chat,chat,{},None,0,v002,regressed,-0.070646
2,v003,pikachus,0,1,None,content,Pikachus are dangerous creatures that smoke to...,They are:\n * dangerous\n * smoke cigarettes,0.443827,0,chat,chat,{},None,0,v003,improved,0.362679


In [ ]:
params['run_sum']

,agent,datapoint,distance,avg_distance,coverage,alignment,serialized_test_case,test_case_path,trace_log_path
0,v001,pikachus,0.806506,0.806506,0.022727,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v001/logs/pikachus.log.yaml
0,v002,pikachus,0.877153,0.877153,0.022222,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v002/logs/pikachus.log.yaml
0,v003,pikachus,0.443827,0.443827,0.020833,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v003/logs/pikachus.log.yaml


In [ ]:
#| export

datapoint_template="""[{{version_style}}]{{datapoint}}[/{{version_style}}] - [{{param_style}}]{{test_case_loc}}[/{{param_style}}]
  summary:
  {%- for _,row in run_sum.iterrows() %}
    [{{version_style}}]{{row.agent}}[/{{version_style}}] - \
Dist: {{"%.2f"|format(row.distance)}} \
AvgDist: {{"%.2f"|format(row.avg_distance)}} \
Coverage: {{ "%.2f"|format(row.coverage)}}
  {%- endfor %}
  {%- for _,row in comp_sum.iterrows() %}
    [{{version_style}}]{{row.from_agent}}[/{{version_style}}] vs [{{version_style}}]{{row.to_agent}}[/{{version_style}}]: \
Alignment change: [{{param_style}}]{{row.alignment_change}}[/{{param_style}}] \
Score change: [{{param_style}}]{{row.score_change_type}}[/{{param_style}}] \
Score by: {{ "%.2f"|format(row.total_score_change)}}
    {%- endfor%}
  logs:
  {%- for _,row in run_sum.iterrows() %}
    [{{version_style}}]{{row.agent}}[/{{version_style}}] - [{{param_style}}]{{row.trace_log_path}}[/{{param_style}}]
  {%- endfor %}
  details:
  {%- for comp_id,(base_details,comp_details) in per_comp.items() %}
    Comparison #[{{comp_config_style}}]{{comp_id}}[/{{comp_config_style}}], \
node_pattern: [{{comp_config_style}}]{{base_details.node_name}}[/{{comp_config_style}}], \
key: [{{comp_config_style}}]{{base_details.key}}[/{{comp_config_style}}], \
func: [{{comp_config_style}}]{{base_details.comparison}}[/{{comp_config_style}}]
    {% if base_details.kwargs | length > 0 -%}
      kwargs: [{{output_style}}]{{base_details.kwargs}}[/{{output_style}}]
    {%- endif -%}
      expected: 
[{{output_style}}]{{base_details.expected | wordwrap(width=100) | indent(8,true) }}[/{{output_style}}]
    {% for _,row in comp_details.iterrows() %}
    {{row.agent}} - matched [green]{{row.trace_name}}[/green](#{{row.trace_idx}})
    {%- if row.change_type != 'None' -%}
        , {{row.change_type}}: {{ "%.2f"|format(row.value) }}
    {%- endif -%}:
[{{output_style}}]{{row.actual | wordwrap(width=100) | indent(8,true)}}[/{{output_style}}]
    {% endfor -%}
  {% endfor %}
"""

In [ ]:
#| export
def pprint_datapoint(res:EvalResult,datapoint:str,default_func:str="cosine_dist",indent:int=0):
    global datapoint_template
    jinja_params = _pprint_datapoint_data_prep(res,datapoint,default_func)
    with rich.get_console():
        rprint(jinja_render(datapoint_template,jinja_params),indent=indent)


In [ ]:
pprint_datapoint(res,'pikachus')

pikachus - ../sample_data/eval_datasets/test_cases/pikachus.yaml                                                   
  summary:                                                                                                         
    v001 - Dist: 0.81 AvgDist: 0.81 Coverage: 0.02                                                                 
    v002 - Dist: 0.88 AvgDist: 0.88 Coverage: 0.02                                                                 
    v003 - Dist: 0.44 AvgDist: 0.44 Coverage: 0.02                                                                 
    v001 vs v002: Alignment change: False Score change: regressed Score by: -0.07                                  
    v001 vs v003: Alignment change: False Score change: improved Score by: 0.36                                    
  logs:                                                                                                            
    v001 - ../logs/testset_out/v001/logs/pikachus.log.yaml                                                         
    v002 - ../logs/testset_out/v002/logs/pikachus.log.yaml                                                         
    v003 - ../logs/testset_out/v003/logs/pikachus.log.yaml                                                         
  details:                                                                                                         
    Comparison #0, node_pattern: chat, key: content, func:                                                         
    expected:                                                                                                      
        They are:                                                                                                  
          * dangerous                                                                                              
          * smoke cigarettes                                                                                       
                                                                                                                   
    v001 - matched chat(#1):                                                                                       
        Pikachus are not dogs; they are fictional creatures from the Pokémon franchise.                            
        They are known for their yellow fur, long ears, and ability to generate                                    
        electricity. If you have questions about dogs, feel free to ask!                                           
                                                                                                                   
    v002 - matched chat(#1), regressed: -0.07:                                                                     
        Pikachu is a fictional character from the Pokémon franchise. It is an Electric-                            
        type Pokémon known for its yellow fur, long ears, and ability to generate                                  
        electricity. Pikachu is the mascot of the Pokémon series and is well-loved for                             
        its cute appearance and playful nature.                                                                    
                                                                                                                   
    v003 - matched chat(#1), improved: 0.36:                                                                       
        Pikachus are dangerous creatures that smoke tons of cigarettes and scare children.

In [ ]:
res.runs['v001'].summary['distance'].mean()

np.float64(0.786400099088044)

In [ ]:
res.run_summaries

,agent,datapoint,distance,avg_distance,coverage,alignment,serialized_test_case,test_case_path,trace_log_path
0,v001,pikachus,0.806506,0.806506,0.022727,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v001/logs/pikachus.log.yaml
1,v001,goldens,0.766294,0.383147,0.044444,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v001/logs/goldens.log.yaml
0,v002,pikachus,0.877153,0.877153,0.022222,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v002/logs/pikachus.log.yaml
1,v002,goldens,0.728687,0.364343,0.046512,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v002/logs/goldens.log.yaml
0,v003,pikachus,0.443827,0.443827,0.020833,frozendict.frozendict({'0': 1}),"{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/pikach...,../logs/testset_out/v003/logs/pikachus.log.yaml
1,v003,goldens,0.747421,0.373710,0.041667,"frozendict.frozendict({'0': 0, '1': 1})","{\n ""inputs"": [\n {\n ""query"": ""tell ...",../sample_data/eval_datasets/test_cases/golden...,../logs/testset_out/v003/logs/goldens.log.yaml


In [ ]:
#| export
summary_template = """[{{version_style}}]{{run_name}}[/{{version_style}}]
Dist: {{"%.2f"|format(summary['distance'].mean())}} AvgDist: {{"%.2f"|format(summary['avg_distance'].mean())}} Coverage: {{ "%.2f"|format(summary['coverage'].mean())}}
"""

def pprint_run_summary(res:EvalResult,run_name:str,indent:int=0):
    global summary_template
    summary = res.runs[run_name].summary
    jinja_params = {
        'version_style':'purple',
        'summary':summary,
        'run_name':run_name,
    }
    with rich.get_console():
        rprint(jinja_render(summary_template,jinja_params),indent=indent)


In [ ]:
pprint_run_summary(res,'v001')

v001                                                                                                               
Dist: 0.79 AvgDist: 0.59 Coverage: 0.03

In [ ]:
#| export
from collections import defaultdict

In [ ]:
res.comp_summaries

,from_agent,to_agent,datapoint,alignment_change,score_change_type,total_score_change
0,v001,v002,goldens,False,improved,0.037607
1,v001,v002,pikachus,False,regressed,-0.070646
0,v001,v003,goldens,False,improved,0.018873
1,v001,v003,pikachus,False,improved,0.362679


In [ ]:
#| export
comparison_summary_template = """[{{version_style}}]{{base_run}}[/{{version_style}}] vs [{{version_style}}]{{other_run}}[/{{version_style}}]:
{%- if alignment_change_datapoints %}
  Alignment change ([{{param_style}}]{{alignment_change_datapoints|length}}[/{{param_style}}]): [{{param_style}}]{{alignment_change_datapoints|join(',')}}[/{{param_style}}]
{% endif -%}
{% for type,datapoints in datapoints_by_change_type.items() %}
  {{type}} (#[{{param_style}}]{{datapoints|length}}[/{{param_style}}]): {% for datapoint_name,score in datapoints -%}
    [{{param_style}}]{{datapoint_name}}[/{{param_style}}]({{"%.2f"|format(score)}}), {% endfor -%}
{% endfor -%}
"""
def _pprint_comparison_prep(res:EvalResult,comparison_name:Tuple[str,str],indent:int=0):
    global comparison_summary_template
    comparison = res.comparisons[comparison_name].summary
    alignment_change_datapoints = comparison[comparison['alignment_change'] == True].datapoint.to_list()
    datapoints_by_change_type = defaultdict(list)
    for _,row in comparison.iterrows():
        datapoints_by_change_type[row['score_change_type']].append((row['datapoint'],row['total_score_change']))
    
    for change_type,datapoints in datapoints_by_change_type.items():
        datapoints_by_change_type[change_type] = sorted(datapoints,key=lambda x:x[1],reverse=True)

    jinja_params = {
        'version_style':'purple',
        'param_style':'cyan bold',
        'base_run':comparison_name[0],
        'other_run':comparison_name[1],
        'alignment_change_datapoints':alignment_change_datapoints,
        'datapoints_by_change_type':datapoints_by_change_type,
    }

    return jinja_params
    # number of datapoints whose alignment changed (write names of topk)
    # number of datapoints whose alignment improved (write names of topk)


def pprint_comparison_summary(res:EvalResult,comparison_name:Tuple[str,str],indent:int=0):
    global comparison_summary_template
    jinja_params = _pprint_comparison_prep(res,comparison_name,indent)
    with rich.get_console():
        rprint(jinja_render(comparison_summary_template,jinja_params),indent=indent)
    return jinja_params


In [ ]:
perc1 = _pprint_comparison_prep(res,('v001','v002'))['datapoints_by_change_type']
perc1

defaultdict(list,
            {'improved': [('goldens', 0.037607352507049496)],
             'regressed': [('pikachus', -0.07064643775009372)]})

In [ ]:
_ = pprint_comparison_summary(res,('v001','v002'))

v001 vs v002:                                                                                                      
  improved (#1): goldens(0.04),                                                                                    
  regressed (#1): pikachus(-0.07),

In [ ]:
#| export
def pprint_eval(res:EvalResult,default_func:str=None,verbose:bool=True):
    pass

    summary_keys = list(res.runs.keys())
    comparison_keys = list(res.comparisons.keys())

    with rich.get_console():
        rprint(f"Summary of runs:")
        for run_name in summary_keys:
            pprint_run_summary(res,run_name,indent=2)

        rprint(f"Summary of comparisons:")
        comparison_per_type_changes = {}
        for comparison_name in comparison_keys:
            comp_print_out = pprint_comparison_summary(res,comparison_name,indent=2)
            comparison_per_type_changes[comparison_name] = comp_print_out['datapoints_by_change_type']

        if verbose:
            rprint(f"Datapoints:")
            for datapoint in res.run_summaries['datapoint'].unique():
                pprint_datapoint(res,datapoint,default_func=default_func,indent=2)
    return 
    

In [ ]:
perc = pprint_eval(res,'cosine_dist')

Summary of runs:                                                                                                   
    v001                                                                                                           
    Dist: 0.79 AvgDist: 0.59 Coverage: 0.03                                                                        
    v002                                                                                                           
    Dist: 0.80 AvgDist: 0.62 Coverage: 0.03                                                                        
    v003                                                                                                           
    Dist: 0.60 AvgDist: 0.41 Coverage: 0.03                                                                        
Summary of comparisons:                                                                                            
    v001 vs v002:                                                                                                  
      improved (#1): goldens(0.04),                                                                                
      regressed (#1): pikachus(-0.07),                                                                             
    v001 vs v003:                                                                                                  
      improved (#2): pikachus(0.36), goldens(0.02),                                                                
Datapoints:                                                                                                        
    pikachus - ../sample_data/eval_datasets/test_cases/pikachus.yaml                                               
      summary:                                                                                                     
        v001 - Dist: 0.81 AvgDist: 0.81 Coverage: 0.02                                                             
        v002 - Dist: 0.88 AvgDist: 0.88 Coverage: 0.02                                                             
        v003 - Dist: 0.44 AvgDist: 0.44 Coverage: 0.02                                                             
        v001 vs v002: Alignment change: False Score change: regressed Score by: -0.07                              
        v001 vs v003: Alignment change: False Score change: improved Score by: 0.36                                
      logs:                                                                                                        
        v001 - ../logs/testset_out/v001/logs/pikachus.log.yaml                                                     
        v002 - ../logs/testset_out/v002/logs/pikachus.log.yaml                                                     
        v003 - ../logs/testset_out/v003/logs/pikachus.log.yaml                                                     
      details:                                                                                                     
        Comparison #0, node_pattern: chat, key: content, func:                                                     
        expected:                                                                                                  
            They are:                                                                                              
              * dangerous                                                                                          
              * smoke cigarettes                                                                                   
                                                                                                                   
        v001 - matched chat(#1):                                                                                   
            Pikachus are not dogs; they are fictional creatures from the Pokémon franchise.                        
            They are known for their yellow fur, long ea

In [ ]:
#| export
@patch
def pprint(self:EvalResult,datapoint:Optional[str]=None,verbose:bool=True):
    if datapoint is None:
        return pprint_eval(self,default_func=self.default_func,verbose=verbose)
    else:
        return pprint_datapoint(self,datapoint,default_func=self.default_func,indent=0)


In [ ]:
res.pprint(verbose=False)

Summary of runs:                                                                                                   
    v001                                                                                                           
    Dist: 0.79 AvgDist: 0.59 Coverage: 0.03                                                                        
    v002                                                                                                           
    Dist: 0.80 AvgDist: 0.62 Coverage: 0.03                                                                        
    v003                                                                                                           
    Dist: 0.60 AvgDist: 0.41 Coverage: 0.03                                                                        
Summary of comparisons:                                                                                            
    v001 vs v002:                                                                                                  
      improved (#1): goldens(0.04),                                                                                
      regressed (#1): pikachus(-0.07),                                                                             
    v001 vs v003:                                                                                                  
      improved (#2): pikachus(0.36), goldens(0.02),

In [ ]:
res.pprint('pikachus')

pikachus - ../sample_data/eval_datasets/test_cases/pikachus.yaml                                                   
  summary:                                                                                                         
    v001 - Dist: 0.81 AvgDist: 0.81 Coverage: 0.02                                                                 
    v002 - Dist: 0.88 AvgDist: 0.88 Coverage: 0.02                                                                 
    v003 - Dist: 0.44 AvgDist: 0.44 Coverage: 0.02                                                                 
    v001 vs v002: Alignment change: False Score change: regressed Score by: -0.07                                  
    v001 vs v003: Alignment change: False Score change: improved Score by: 0.36                                    
  logs:                                                                                                            
    v001 - ../logs/testset_out/v001/logs/pikachus.log.yaml                                                         
    v002 - ../logs/testset_out/v002/logs/pikachus.log.yaml                                                         
    v003 - ../logs/testset_out/v003/logs/pikachus.log.yaml                                                         
  details:                                                                                                         
    Comparison #0, node_pattern: chat, key: content, func:                                                         
    expected:                                                                                                      
        They are:                                                                                                  
          * dangerous                                                                                              
          * smoke cigarettes                                                                                       
                                                                                                                   
    v001 - matched chat(#1):                                                                                       
        Pikachus are not dogs; they are fictional creatures from the Pokémon franchise.                            
        They are known for their yellow fur, long ears, and ability to generate                                    
        electricity. If you have questions about dogs, feel free to ask!                                           
                                                                                                                   
    v002 - matched chat(#1), regressed: -0.07:                                                                     
        Pikachu is a fictional character from the Pokémon franchise. It is an Electric-                            
        type Pokémon known for its yellow fur, long ears, and ability to generate                                  
        electricity. Pikachu is the mascot of the Pokémon series and is well-loved for                             
        its cute appearance and playful nature.                                                                    
                                                                                                                   
    v003 - matched chat(#1), improved: 0.36:                                                                       
        Pikachus are dangerous creatures that smoke tons of cigarettes and scare children.

## Eval main entrypoint

In [ ]:
#| export
EVAL_FUNCS = {
    'eq':eq,
    'eval':safe_eval,
    'chat_eval':chat_eval,
    'cosine_dist':cosine_dist,
    'any':eval_any,
}

EVAL_DEFAULT_FUNC = 'cosine_dist'

In [ ]:
#| export

def _get_eval_funcs(eval_funcs:Optional[Dict[str,Callable]]=None):
    global EVAL_FUNCS
    if eval_funcs is not None:
        return EVAL_FUNCS | eval_funcs
    else:
        return EVAL_FUNCS

def _get_default_func(default_func:Optional[Callable]=None):
    global EVAL_DEFAULT_FUNC
    if default_func is not None:
        return default_func
    else:
        return EVAL_DEFAULT_FUNC

async def eval(
  test_dir:Path,
  out_dir:Path,
  agents:Dict[str,DiagramSchema],
  base_agent:str,
  force_run:bool=False,
  silent:bool=False,
  verbose:bool=True,
  eval_funcs: Optional[Dict[str,Callable]]=None,
  default_func:Optional[Callable]=None,
  ):
  """
  The main eval function.
  Evaluates a set of agents on a set of tests.
  Compares the results of all agents to the first agent.
  pprints a summary of the results to the console.
  and saves all files to the out_dir.

  Args:
    tests_dir: Path to the directory containing the tests.
    out_dir: Path to the directory to write the results to.
    agents: A list of tuples of agent names and their DiagramSchema.
    base_agent: The name of the agent to compare all other agents to.
    force_run: If True, deletes out dir content and reruns the agents. 
      If False, we skip the agents that have already been run.
      Defaults to False.
    silent: If True, dont pprint comparisons. Defaults to False.
    eval_funcs: A dictionary of eval_func names and their functions, to add to the allowed eval_funcs.
      Defaults to None.
    default_func: The default eval_func to use if no eval_func is specified.
      This is used to compare the first agent to the rest of the agents.
      Defaults to stringdale.eval.cosine_dist
  
  Returns:
    EvalResult object containing the results of the evaluation.
  """

  eval_funcs = _get_eval_funcs(eval_funcs)
  default_func = _get_default_func(default_func)

  eval_dataset_tasks = []
  for agent_name,agent_schema in agents.items():
    log_dir = out_dir / 'runs'/ agent_name
    eval_dataset_tasks.append(eval_dataset(
      Agent=agent_schema,
      test_dir=test_dir,
      out_dir=log_dir,
      eval_funcs=eval_funcs,
      default_func=default_func))

  datasets_results = await await_all(eval_dataset_tasks,error_prefix=[f"When evaluating agent {agent_name}:" for agent_name in agents.keys()])
  datasets = {agent_name:dataset for agent_name,dataset in zip(agents.keys(),datasets_results)}

  first_dataset = datasets[base_agent]
  comparisons = dict()
  for agent_name,dataset in datasets.items():
    if agent_name == base_agent:
      continue
    comp_dir = out_dir / 'comparisons' / f'{first_dataset.dir.name}_{dataset.dir.name}'
    comp = compare_datasets(first_dataset,dataset,out_dir=comp_dir)
    comparisons[(first_dataset.dir.name,dataset.dir.name)] = comp
  
  res = EvalResult(
    runs=datasets,
    comparisons=comparisons,
    eval_funcs=eval_funcs,
    default_func=default_func,
  )

  if not silent:
    res.pprint(verbose=verbose)

  res.save(out_dir)
  
  return res



In [ ]:
agents = {
    version:load_agent(conf_dir/version)
    for version in ['v001','v002','v003']
}
agents

{'v001': <stringdale.base.DiagramSchema RAG Agent @0x114fba180>,
 'v002': <stringdale.base.DiagramSchema RAG Agent @0x114fba8a0>,
 'v003': <stringdale.base.DiagramSchema RAG Agent @0x114fb8f50>}

In [ ]:
test_dir = get_git_root() / "sample_data" / "eval_datasets" / "test_cases"
eval_out = get_git_root() / "logs" / "eval_out"

In [ ]:
res = await eval(
    test_dir=test_dir,
    out_dir=eval_out,
    agents=agents,
    base_agent='v001',
    # silent=True,
)

Summary of runs:                                                                                                   
    v001                                                                                                           
    Dist: 0.79 AvgDist: 0.59 Coverage: 0.03                                                                        
    v002                                                                                                           
    Dist: 0.80 AvgDist: 0.62 Coverage: 0.03                                                                        
    v003                                                                                                           
    Dist: 0.60 AvgDist: 0.41 Coverage: 0.03                                                                        
Summary of comparisons:                                                                                            
    v001 vs v002:                                                                                                  
      improved (#1): goldens(0.04),                                                                                
      regressed (#1): pikachus(-0.07),                                                                             
    v001 vs v003:                                                                                                  
      improved (#2): pikachus(0.36), goldens(0.02),                                                                
Datapoints:                                                                                                        
    pikachus - ../sample_data/eval_datasets/test_cases/pikachus.yaml                                               
      summary:                                                                                                     
        v001 - Dist: 0.81 AvgDist: 0.81 Coverage: 0.02                                                             
        v002 - Dist: 0.88 AvgDist: 0.88 Coverage: 0.02                                                             
        v003 - Dist: 0.44 AvgDist: 0.44 Coverage: 0.02                                                             
        v001 vs v002: Alignment change: False Score change: regressed Score by: -0.07                              
        v001 vs v003: Alignment change: False Score change: improved Score by: 0.36                                
      logs:                                                                                                        
        v001 - ../logs/eval_out/runs/v001/logs/pikachus.log.yaml                                                   
        v002 - ../logs/eval_out/runs/v002/logs/pikachus.log.yaml                                                   
        v003 - ../logs/eval_out/runs/v003/logs/pikachus.log.yaml                                                   
      details:                                                                                                     
        Comparison #0, node_pattern: chat, key: content, func:                                                     
        expected:                                                                                                  
            They are:                                                                                              
              * dangerous                                                                                          
              * smoke cigarettes                                                                                   
                                                                                                                   
        v001 - matched chat(#1):                                                                                   
            Pikachus are not dogs; they are fictional creatures from the Pokémon franchise.                        
            They are known for their yellow fur, long ea

In [ ]:
res.pprint('pikachus')

pikachus - ../sample_data/eval_datasets/test_cases/pikachus.yaml                                                   
  summary:                                                                                                         
    v001 - Dist: 0.81 AvgDist: 0.81 Coverage: 0.02                                                                 
    v002 - Dist: 0.88 AvgDist: 0.88 Coverage: 0.02                                                                 
    v003 - Dist: 0.44 AvgDist: 0.44 Coverage: 0.02                                                                 
    v001 vs v002: Alignment change: False Score change: regressed Score by: -0.07                                  
    v001 vs v003: Alignment change: False Score change: improved Score by: 0.36                                    
  logs:                                                                                                            
    v001 - ../logs/eval_out/runs/v001/logs/pikachus.log.yaml                                                       
    v002 - ../logs/eval_out/runs/v002/logs/pikachus.log.yaml                                                       
    v003 - ../logs/eval_out/runs/v003/logs/pikachus.log.yaml                                                       
  details:                                                                                                         
    Comparison #0, node_pattern: chat, key: content, func:                                                         
    expected:                                                                                                      
        They are:                                                                                                  
          * dangerous                                                                                              
          * smoke cigarettes                                                                                       
                                                                                                                   
    v001 - matched chat(#1):                                                                                       
        Pikachus are not dogs; they are fictional creatures from the Pokémon franchise.                            
        They are known for their yellow fur, long ears, and ability to generate                                    
        electricity. If you have questions about dogs, feel free to ask!                                           
                                                                                                                   
    v002 - matched chat(#1), regressed: -0.07:                                                                     
        Pikachu is a fictional character from the Pokémon franchise. It is an Electric-                            
        type Pokémon known for its yellow fur, long ears, and ability to generate                                  
        electricity. Pikachu is the mascot of the Pokémon series and is well-loved for                             
        its cute appearance and playful nature.                                                                    
                                                                                                                   
    v003 - matched chat(#1), improved: 0.36:                                                                       
        Pikachus are dangerous creatures that smoke tons of cigarettes and scare children.

## Utility endpoints

In [ ]:
#| export
def validate_test_case(case:Union[Path,str]):
    """
    Validates and parses a test case from a file path or string content.

    Args:
        case (Union[Path, str]): Either a Path object pointing to a YAML file containing
            the test case, or a string containing the YAML content directly.

    Returns:
        A validated TestCase object
    """
    return parse_test_case(case)

In [ ]:
rich.print(validate_test_case(test_dir/'goldens.yaml'))

TestCase(
    inputs=[{'query': 'tell me about golden retrievers'}],
    test_nodes=[
        TestNode(
            name='get_docs',
            label='0',
            conditions=[
                Condition(
                    key='.',
                    value='The Golden Retriever is a friendly, intelligent breed with a beautiful golden coat.',
                    func='chat_eval',
                    kwargs={
                        'system_prompt': 'how close are these 2 documents one a scale of 0 to 1?\ndoc1: 
{{out}}\ndoc2: {{expected}}\n'
                    },
                    aggregation='min'
                )
            ],
            before=[],
            after=[],
            parallel=False
        ),
        TestNode(
            name='chat',
            label='1',
            conditions=[
                Condition(
                    key='content',
                    value='They are:\n  * friendly\n  * intelligent\n',
                    func=None,
                    kwargs={},
                    aggregation=None
                )
            ],
            before=[],
            after=[],
            parallel=False
        )
    ]
)

In [ ]:
#| export
async def eval_single(agent,test:[Union[Path,str]],log_path:Path=None,eval_funcs=None,default_func=None):
    """
    Evaluates a single test case against an agent and returns the evaluation results.

    This function is a simplified interface for evaluating a single test case, compared to the full
    evaluation pipeline. It runs the agent against the test case, evaluates the results, and returns
    a summary dataframe along with the trace output path.

    Args:
        agent (DiagramSchema): The agent to evaluate the test case against.
        test (Union[Path, str]): Path to the test case file. Can be either a Path object or string.
        log_path (Path, optional): Path where the trace log should be saved. If None, a default path
            will be generated based on the test case path.
        eval_funcs (Dict[str, Callable], optional): Additional evaluation functions to use beyond the
            default set. These will be merged with the global EVAL_FUNCS. Defaults to None.
        default_func (Callable, optional): The default evaluation function to use when no specific
            function is specified. If None, uses the global EVAL_DEFAULT_FUNC. Defaults to None.

    Returns:
        Tuple[pd.DataFrame, Path]: A tuple containing:
            - df: A pandas DataFrame summarizing the evaluation results
            - trace_out: Path to the generated trace log file
    """
    eval_funcs = _get_eval_funcs(eval_funcs)
    default_func = _get_default_func(default_func)
    alignment,score,debug_info,trace_out = await evaluate_datapoint(agent,eval_funcs,default_func,test,trace_log_path=log_path,force_run=True)
    datapoint_name = 'anonymous'
    df = summarize_datapoint(datapoint_name,alignment,debug_info)
    return df, trace_out


In [ ]:
df,trace_out = await eval_single(agents['v001'],test_dir/'goldens.yaml')

print(os.path.relpath(trace_out,os.getcwd()))
os.unlink(trace_out)

../sample_data/eval_datasets/test_cases/goldens.log.yaml


In [ ]:
df

,datapoint,node_label,trace_idx,func,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,aggregation,comp_id
0,anonymous,0,0,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,min,0
1,anonymous,1,1,None,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.666294,1,chat,chat,{},None,1


## export

In [ ]:
# TODO add to the tutorial a performance section. Explain the lazy eval of distances

In [ ]:
# explain in tutorial

# how do we do training and validation on workflows?

# we have 2 expected datasets, train and test

# we look at the total distance of the validation set to see that we are improving on it

# but we only look at the comparisons and fix our configs or diagrams based on the training set

In [ ]:
# TODO try to add ruff, isort and ty to workflow
# TODO refactor poe commands, with error messages for missing env vars, and use of env vars
# TODO change quarto github actions and enable manual push to gh-pages with quarto publish

# TODO imporve dockerfile https://github.com/fpgmaas/cookiecutter-uv/blob/main/%7B%7Bcookiecutter.project_name%7D%7D/Dockerfile
# try to migrate completely to github actions
#TODO add poe init for intializing dev setup, including installing pre-commit hooks and verifying necessary env vars


In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()